In [ ]:
import pandas as pd
df = pd.read_csv('combined_data_filtered.csv')


In [3]:
# Select only rows with revenue and where event_name is filled
from time import localtime, strftime

#revenue_data = df.loc[(~df['{reporting_revenue}'].isna()) & (~df['{event_name}'].isna()), ['{adid}', '{random_user_id}','{country}', '{created_at}','{installed_at}', '{network_name}', '{store}',\
#    '{tracker}', '{campaign_name}','{adgroup_name}', '{creative_name}', '{activity_kind}', '{event_name}', '{subscription_event_type}', '{subscription_event_subtype}','{currency}', '{revenue}', '{reporting_currency}', '{reporting_revenue}']]

df = df.sort_values('{created_at}')

# Make dates (provided as epochs) readable
df['event_date']   = df['{created_at}'].apply(lambda x:strftime("%Y-%m-%d", localtime(x)))
df['install_date'] = df['{installed_at}'].apply(lambda x:strftime("%Y-%m-%d", localtime(x)))

# Find the first payment of every user
first_payment_per_user = df.groupby('{random_user_id}').first()[['{created_at}', '{subscription_event_type}']]
first_payment_per_user = first_payment_per_user.rename({
                                                '{created_at}':'first_payment_date', 
                                                '{event_name}':'first_payment_event_name'}, axis=1)

# Rank of event_name per day ==> there should be only one event of a kind per day and per user_id. However, there are doubles. 
event_rank = df[['event_date','{random_user_id}','{event_name}', '{currency}']].groupby(['event_date','{random_user_id}','{event_name}']).rank(method='first')
event_rank.rename(columns={'{currency}':'rank'}, inplace=True)
df['rank'] = event_rank
#revenue_data = revenue_data.merge(event_rank, how='inner', left_index=True, right_index=True)

# Merge it back to the dataframe
revenue_data = df.merge(first_payment_per_user, how='left', left_on='{random_user_id}', right_index=True).sort_values(['{random_user_id}','{created_at}'])

In [4]:
def get_quarter(date_str):
    year, month, _ = map(int, date_str.split('-'))
    quarter = (month - 1) // 3 + 1
    return f"Q{quarter}-{year}"

# Example usage:
date_str = "2024-02-09"
quarter = get_quarter(date_str)
print(f"The quarter for {date_str} is {quarter}")

The quarter for 2024-02-09 is Q1-2024


In [5]:
from math import floor

# Compute the time elapsed in seconds since the first payment
revenue_data['time_since_first_payment'] = revenue_data['{created_at}'] - revenue_data['first_payment_date']

# Compute the event date year
revenue_data['event_date_year'] = revenue_data['event_date'].apply(lambda x:int(x[:4]))
revenue_data['event_date_quarter'] = revenue_data['event_date'].apply(get_quarter)

# Convert it in months
revenue_data['i_th_month_since_first_payment'] = revenue_data['time_since_first_payment'] /(30*24*3600)
revenue_data['i_th_month_since_first_payment'] = revenue_data['i_th_month_since_first_payment'].apply(lambda x:1+floor(x)).astype(int)

# Compute net revenue (70% of the reporting revenue)
revenue_data['net_revenue'] = revenue_data['{reporting_revenue}'] * 0.7

# Define the ratio

# Default ratio is 0
revenue_data['revenue_deal_ratio'] = 0.0

# For hozana : 50% of the net revenue in the first 12 months
revenue_data.loc[(revenue_data['{campaign_name}']=='hozana') & \
                 (~revenue_data['{adgroup_name}'].isin(['cp-semaine-sainte', 'CP', 'community'])) & \
                 (revenue_data['i_th_month_since_first_payment'] <= 12), 'revenue_deal_ratio'] = 0.5

revenue_data.loc[(revenue_data['{campaign_name}']=='Hozana') & \
                 (~revenue_data['{adgroup_name}'].isin(['CP-rentree', 'CP', 'cdi-mdj'])) & \
                 (revenue_data['i_th_month_since_first_payment'] <= 12), 'revenue_deal_ratio'] = 0.5

revenue_data.loc[(revenue_data['{campaign_name}']=='Hozana_affilie') & \
                 (~revenue_data['{adgroup_name}'].isin(['cp-semaine-sainte', 'CP', 'community'])) & \
                 (revenue_data['i_th_month_since_first_payment'] <= 12), 'revenue_deal_ratio'] = 0.5

revenue_data.loc[(revenue_data['{campaign_name}']=='Icnews') & \
                 (revenue_data['i_th_month_since_first_payment'] <= 12), 'revenue_deal_ratio'] = 0.5

revenue_data.loc[(revenue_data['{campaign_name}']=='Icnews') & \
                 (revenue_data['i_th_month_since_first_payment'] >= 13) & \
                 (revenue_data['i_th_month_since_first_payment'] <= 36), 'revenue_deal_ratio'] = 0.25

# For other deals
# ...
# ...

# Compute the deal amount
revenue_data['net_revenue_deal_amount'] = revenue_data['net_revenue'] * revenue_data['revenue_deal_ratio']

In [11]:
revenue_data[revenue_data['{campaign_name}'].isin(['Icnews'])]

,{reporting_revenue},{event_name},{created_at},{installed_at},{random_user_id},{subscription_event_type}_x,{currency},{campaign_name},{tracker},{adgroup_name},...,rank,first_payment_date,{subscription_event_type}_y,time_since_first_payment,event_date_year,event_date_quarter,i_th_month_since_first_payment,net_revenue,revenue_deal_ratio,net_revenue_deal_amount
113193,0.990414,renewal,1705756721,1697269626,019ad8bb72ecf72ef58409fbd444a70bd2852a1d,NaN,USD,Icnews,15tsp0rc,je_suis,...,1.0,1705756721,NaN,0,2024,Q1-2024,1,0.693290,0.5,0.346645
115309,0.989545,renewal,1706361521,1697269626,019ad8bb72ecf72ef58409fbd444a70bd2852a1d,NaN,USD,Icnews,15tsp0rc,je_suis,...,1.0,1705756721,NaN,604800,2024,Q1-2024,1,0.692682,0.5,0.346341
117425,0.989947,renewal,1706966321,1697269626,019ad8bb72ecf72ef58409fbd444a70bd2852a1d,NaN,USD,Icnews,15tsp0rc,je_suis,...,1.0,1705756721,NaN,1209600,2024,Q1-2024,1,0.692963,0.5,0.346482
119569,0.990303,renewal,1707571121,1697269626,019ad8bb72ecf72ef58409fbd444a70bd2852a1d,NaN,USD,Icnews,15tsp0rc,je_suis,...,1.0,1705756721,NaN,1814400,2024,Q1-2024,1,0.693212,0.5,0.346606
121676,0.990080,renewal,1708175921,1697269626,019ad8bb72ecf72ef58409fbd444a70bd2852a1d,NaN,USD,Icnews,15tsp0rc,je_suis,...,1.0,1705756721,NaN,2419200,2024,Q1-2024,1,0.693056,0.5,0.346528
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
159041,1.205886,renewal,1718252346,1698291454,fec3f21c0b7936f37cc571f58471d10af32bff8c,NaN,USD,Icnews,15eoxbri,UMCJ,...,1.0,1698920395,NaN,19331951,2024,Q2-2024,8,0.844120,0.5,0.422060
159571,2.989425,renewal,1718377678,1697283496,fed746302b44f552fb85c73c6c3c91a8ca34986d,NaN,USD,Icnews,15tsp0rc,je_suis,...,1.0,1718377678,NaN,0,2024,Q2-2024,1,2.092597,0.5,1.046299
161786,2.990387,renewal,1718970151,1697283496,fed746302b44f552fb85c73c6c3c91a8ca34986d,NaN,USD,Icnews,15tsp0rc,je_suis,...,1.0,1718377678,NaN,592473,2024,Q2-2024,1,2.093271,0.5,1.046635
164067,2.989621,renewal,1719587278,1697283496,fed746302b44f552fb85c73c6c3c91a8ca34986d,NaN,USD,Icnews,15tsp0rc,je_suis,...,1.0,1718377678,NaN,1209600,2024,Q2-2024,1,2.092735,0.5,1.046367


In [14]:
! uv add --active gspread gspread-dataframe

Resolved 29 packages in 1ms
Installed 28 packages in 1.64s
 + boto3==1.42.28
 + botocore==1.42.28
 + certifi==2026.1.4
 + charset-normalizer==3.4.4
 + colorama==0.4.6
 + google-auth==2.47.0
 + google-auth-oauthlib==1.2.2
 + gspread==6.2.1
 + gspread-dataframe==4.0.0
 + idna==3.11
 + jmespath==1.0.1
 + numpy==2.4.1
 + oauthlib==3.3.1
 + pandas==2.3.3
 + pyasn1==0.6.1
 + pyasn1-modules==0.4.2
 + python-dateutil==2.9.0.post0
 + pytz==2025.2
 + requests==2.32.5
 + requests-oauthlib==2.0.0
 + rsa==4.9.1
 + s3transfer==0.16.0
 + six==1.17.0
 + smart-open==7.5.0
 + tqdm==4.67.1
 + tzdata==2025.3
 + urllib3==2.6.3
 + wrapt==2.0.1


In [21]:

import gspread
from gspread_dataframe import set_with_dataframe

# gc = gspread.oauth()
gc = gspread.service_account()

#sh_hoz = gc.open_by_url('https://docs.google.com/spreadsheets/d/1bb8B_8ttZv7XjM0JNF111aiKiLjTzLkSD0KTFPsEPGw')
#worksheet = sh_hoz.worksheet('Données brutes')
#worksheet.clear()
#set_with_dataframe(worksheet, revenue_data[revenue_data['{campaign_name}'].isin(['hozana', 'Hozana', 'Hozana_affilie'])])

sh_hoz = gc.open_by_url('https://docs.google.com/spreadsheets/d/1fM2Uz3udpnVQvON79NNLO8xDKiewwTUa8KILSlWoov4')
worksheet = sh_hoz.worksheet('Données brutes')
worksheet.clear()
set_with_dataframe(worksheet, revenue_data[revenue_data['{campaign_name}'].isin(['Icnews'])])